In [18]:
!pip install fasttext
!pip install catboost
!pip install huggingface_hub
!pip install polars
!pip install polars-u64-idx
!pip install scikit-learn
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 117.0 MB/s eta 0:00:00a 0:00:01
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defa

### Load embeddings for english words

In [2]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-en-vectors", filename="model.bin")
model = fasttext.load_model(model_path)
model.words[:10]

/home/student/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[',', 'the', '.', 'and', 'to', 'of', 'a', '</s>', 'in', 'is']

### Load dataset and split train and test

In [3]:
import polars as pl
import numpy as np
import random
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric

random_state = 42
random.seed(random_state)
np.random.seed(random_state)

In [5]:
df = pl.read_parquet("../dataset/medium_articles.parquet")
df.select(pl.len())

len
u64
100000


In [6]:
df.select("*").limit(5)

text,tags,label_tags
str,list[str],"array[f32, 43564]"
"""Photo by Josh Riemer on Unspla…","[""Mental Health"", ""Health"", … ""Neuroscience""]","[0.0, 0.0, … 0.0]"
"""Your Brain On Coronavirus A g…","[""Mental Health"", ""Coronavirus"", … ""Neuroscience""]","[0.0, 0.0, … 0.0]"
"""Mind Your Nose How smell trai…","[""Biotechnology"", ""Neuroscience"", … ""Science""]","[0.0, 0.0, … 0.0]"
"""Passionate about the synergy b…","[""Health"", ""Neuroscience"", … ""Science""]","[0.0, 0.0, … 0.0]"
"""You’ve heard of him, haven’t y…","[""Brain"", ""Health"", … ""Science""]","[0.0, 0.0, … 0.0]"


### Get embeddings from fasttext

In [7]:
X = [model.get_sentence_vector(s.replace("\n", "")) for s in df["text"]]

In [12]:
X[0]

array([-1.48390243e-02,  1.38898916e-03,  5.38585754e-03,  4.38489988e-02,
       -3.57368737e-02, -1.87596399e-02,  4.36317967e-03, -4.07333137e-04,
       -1.27658510e-04,  9.22910229e-04, -5.66016417e-03, -5.85928140e-03,
       -3.45777953e-03,  7.80685456e-04, -1.23144602e-02, -7.96648022e-03,
        2.29168031e-02,  1.09916311e-02, -2.12812573e-02,  2.49335151e-02,
       -3.60213732e-03,  1.35593889e-02,  1.57159567e-03, -1.01784710e-02,
       -2.43379846e-02, -2.41253488e-02, -6.70705689e-04, -1.30754225e-02,
       -5.22141205e-03,  5.51616326e-02,  6.72938651e-04, -1.32791465e-02,
        2.89224205e-03, -1.74348839e-02,  5.29893348e-03, -1.93999615e-03,
        1.10751751e-03,  2.07846444e-02, -2.75695021e-03, -8.18019360e-03,
       -1.41886203e-03,  4.38753236e-03, -6.16181828e-03,  3.62516269e-02,
        1.50966912e-03,  1.38033433e-02,  2.84453901e-03, -5.34708321e-04,
       -8.48041847e-03,  8.54781363e-03,  3.84093425e-03, -3.51673830e-03,
       -6.68130815e-03, -

In [ ]:
Y = list(df.select("label_tags")["label_tags"])
Y = [y.to_list() for y in Y]
Y[0]

### Train catboost model on embeddings

In [ ]:
Y = Y
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=random_state)
train_pool = Pool(X_train, Y_train)
test_pool = Pool(X_test, Y_test)

In [ ]:
%%time
clf = CatBoostClassifier(
    loss_function='MultiLogloss',
    eval_metric='HammingLoss',
    iterations=500,
    # class_names=['A', 'B', 'C', 'D', 'E']
)
clf.fit(train_pool, eval_set=test_pool, metric_period=10, plot=True, verbose=50)